### Read and split the document

In [134]:
from typing import List

def doc_to_chunks(doc_file: str) -> List[str]:
    # Read document
    content = ""
    try:
        with open(doc_file, 'r') as file:
            content = file.read()
        
        # Separate the document by split
        chunks = content.split("\n\n")
    except:
        print("File not found!")
        
    return chunks


In [135]:
# Read local document
chunks = doc_to_chunks("doc.md")

for i, chunk in enumerate(chunks):
    print(f"[{i}] {chunk}")

[0] # Black Myth Wukong vs. Superman: Clash of Legends
[1] In the twilight between worlds, a rift opened—an anomaly in time and space. From the East came *Wukong*, the Monkey King, reborn in the shadows of Black Myth. From the West came *Superman*, the paragon of justice, summoned by Earth’s last whisper of hope.
[2] ## The Battlefield
[3] The rift’s heart was a land of fusion—skyscrapers twisted with bamboo forests, neon signs flickered beside ancient shrines. Reality itself cracked as both warriors arrived, eyes locked.
[4] ## The First Strike
[5] Wukong vanished into mist. With *Cloud-Stepping Art*, he multiplied into hundreds of phantom clones. Each clone wielded his legendary *Ruyi Jingu Bang*, extending and shrinking at will, striking with unpredictable rhythm.
[6] Superman countered with his heat vision, disintegrating clones in bursts of plasma. He dashed through the chaos at Mach speed, but Wukong anticipated every move. With a flick, Wukong summoned *Fenghou Fire*, an ancient

### Chunk Embed

In [136]:
from sentence_transformers import SentenceTransformer

# Setting the embedding model
model = SentenceTransformer("shibing624/text2vec-base-chinese")

# Embedding Function
def chunk_embed(chunks: str) -> List[float]:
    embedding = model.encode(chunks, normalize_embeddings=True)
    return embedding.tolist()


In [137]:
# Testing embedding function
embedding_test = chunk_embed("Testing")
print(len(embedding_test))
print(embedding_test[0])

768
0.010464044287800789


In [138]:
# Using the real data (document to embed)
embeddings = [chunk_embed(chunk) for chunk in chunks]
print(len(embeddings))
print(embeddings[0])

17
[0.010463006794452667, -0.02390635944902897, 0.05318445712327957, 0.023943431675434113, 0.016948487609624863, -0.09215254336595535, -0.005165456794202328, 0.013839047402143478, -0.03937763720750809, -0.005633515305817127, -0.07244603335857391, 0.011879773810505867, 0.009343370795249939, 0.03356928378343582, -0.018046338111162186, 0.030475005507469177, 0.06849561631679535, 0.04472767561674118, -0.0376158133149147, -0.0090513089671731, 0.01703363098204136, 0.03868838772177696, -0.03237462788820267, 0.018722157925367355, 0.029091142117977142, -0.01718553528189659, -0.0041940538212656975, -0.0027240675408393145, -0.006315140053629875, 0.061471208930015564, -0.003083708230406046, -0.00896301120519638, -0.06450339406728745, 0.03179129213094711, -0.0014671615790575743, 0.047526586800813675, 0.05031369626522064, 0.012389902025461197, -0.026048753410577774, -0.007165560033172369, -0.016601664945483208, 0.05092908814549446, -0.09299007058143616, -0.055597927421331406, -0.024035628885030746, 0

### Using Vector Database (ChromaDB)

In [139]:
import chromadb

# Create an in-memory instance of Chroma (not store in your local)
chromadb_client = chromadb.EphemeralClient()
# Create a collection to collect the data
chromadb_collection = chromadb_client.get_or_create_collection(name='rag_doc')

def save_embedding_data(chunks: List[str], embeded_chunks: List[List[float]]) -> None:
    for i, (chunk, embeded_chunk) in enumerate(zip(chunks, embeded_chunks)):
        chromadb_collection.add(
            ids=[str(i)],
            documents=[chunk],
            embeddings=[embeded_chunk]
        )

In [140]:
# Operate the data included document and embedding numbers
save_embedding_data(chunks, embeddings)

### Before the above blocks, we call setup the environment.
- Read Document
- Split Document into Chunk
- Embedding Each Chunk
- Save Chunks to the Vector DB

### After here, we start to build the document
- Retrieve
    - By send the query to vector DB, vectorDB can compared the relationship of size and pick up the most top K chunks from the db.
- Rank (Cross Encode)

#### Retrieve

In [141]:
def retrieve(query:str, top_k: int) -> List[str]:
    # in order to compared with chunks, we need to embed the query first.
    query_embedding = chunk_embed(query)
    results = chromadb_collection.query(
        query_embeddings = [query_embedding],
        n_results = top_k
    )
    #?? why the key is documents
    return results['documents'][0]

In [142]:
# Testing the retrieving function

query = "What strategy did Black Myth Wukong use to defeat Superman?"
retrieved_chunks = retrieve(query, 5)

for i, chunk in enumerate(retrieved_chunks):
    print(f"[{i}] {chunk}\n")

[0] Superman countered with his heat vision, disintegrating clones in bursts of plasma. He dashed through the chaos at Mach speed, but Wukong anticipated every move. With a flick, Wukong summoned *Fenghou Fire*, an ancient demonic flame that ignored physical resistance, forcing Superman to divert skyward.

[1] # Black Myth Wukong vs. Superman: Clash of Legends

[2] Both warriors were exhausted. Wukong’s divine energy shimmered erratically. Superman’s cape was scorched, eyes dimmed.

[3] Wukong struck first with *Heaven-Crushing Palm*, collapsing the sky above. Superman met it with *Solar Flare Punch*, the distilled essence of a dying star.

[4] Wukong—on one knee, breathing heavily—watched Superman lying unconscious, a faint smile on his face. The Monkey King stood, bowed in silent respect, and vanished into the shadows, leaving behind the tale of a clash that transcended myth and heroism.



#### Rank (Cross encode)

In [143]:
# Using the re-rank function to get the result
from sentence_transformers import CrossEncoder

def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    # load cross-encode model
    cross_encoder_model = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    # Using pair (List[Tuple[str, str]]) to group the retrieved_chunk
    # Each pair (query, chunk_01), (query, chunk_02), ...
    pairs = []
    for chunk in retrieved_chunks:
        pairs.append((query, chunk))
    
    # Calculate each score from pairs list
    scores = cross_encoder_model.predict(pairs)

    # Combined retrieved_chunk with score
    scores_chunks = list(zip(retrieved_chunks, scores))
    
    # sorted by scores
    scores_chunks.sort(key=lambda x: x[1], reverse=True)
    
    # return scores_chunks[:top_k]
    return [chunk for chunk, _ in scores_chunks][:top_k]
    

In [144]:
rerank_chunk = rerank(query, retrieved_chunks, 3)

for i, chunk in enumerate(rerank_chunk):
    print(f"[{i}] {chunk}\n")

[0] Superman countered with his heat vision, disintegrating clones in bursts of plasma. He dashed through the chaos at Mach speed, but Wukong anticipated every move. With a flick, Wukong summoned *Fenghou Fire*, an ancient demonic flame that ignored physical resistance, forcing Superman to divert skyward.

[1] # Black Myth Wukong vs. Superman: Clash of Legends

[2] Wukong—on one knee, breathing heavily—watched Superman lying unconscious, a faint smile on his face. The Monkey King stood, bowed in silent respect, and vanished into the shadows, leaving behind the tale of a clash that transcended myth and heroism.



### Using OpenAI to response the question

In [155]:
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
client  = OpenAI()

def generateAI(query: str, rerank_chunk: List[str]) -> str:

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                'role': 'system',
                'content':"You are an AI assistant. Use the following documents to answer the user’s question:\n\n" + "\n".join(rerank_chunk)
            },
            {
                'role':'user',
                'content':query
            }
        ]
    )
    return (completion.choices[0].message.content)

In [158]:
query="who is Wukong"

answer = generateAI(query, rerank_chunk)
print(answer)

Wukong, also known as the Monkey King, is a legendary character from Chinese mythology and literature. He is a powerful and mischievous figure known for his incredible strength, agility, and magical abilities. Wukong is the central character in the classic Chinese novel "Journey to the West", where he accompanies a Buddhist monk on a journey to retrieve sacred scriptures. His story has been retold in various forms of media and has become a significant part of Chinese culture and folklore. In the context of the excerpt provided, Wukong is depicted in a battle against Superman, showcasing his formidable skills and abilities.
